# Data Quality Analysis as a Precursor to Machine Learning Activities

A student-information-system (SIS) intends to include additional ML-based insights into its management reporting suite. One of the csv data files used to prime the SIS Machine Learning application has been found to have significant data quality issues.

You have been asked to provide advice and guidance on the steps that need to be taken to clean the data file as part of quality assurance for the ML system.

**To complete this assignment:**

Carry out the following tasks

1. Perform detailed analysis of data quality for the dataset provided, making use of some of the criteria discussed in the week’s lectures.
2. Map out, in outline form, what data cleansing measures would need to be put in place to prepare the dataset for ML work. Indicate the types of Pandas-Scikit-Learn python commands that could be used whilst carrying out the measures you suggested in task 2.
3. Attempt a clean-up of the dataset using the procedures set out in (2) and (3) and report on the degree of success with carrying this out.
4. Submit the final document (in MS-Word or pdf format) containing your responses for sub-tasks 1, 2, 3 and 4 in form of a brief report (500 words). The Python code must be included in your submission.

Please review this document for formatting guidelines.

## Importing and Data Preparation

### Importing

Start by importing the appropriate packages required for this data cleanup as well as the raw data file into a pandas DataFrame.

In [1]:
# Import Packages
import numpy as np
import pandas as pd
import re
from collections import Counter

# Import Data
data = pd.read_csv('source_data\SIS_Faculty-List.csv', encoding='utf-8')

### Initial Data Preparation
Some of the column names are long or contain new lines, so remove these. As a precaution for the columns made of strings, trim all the columns so that any leading / training spaces are removed.

In [2]:
# Data Preparation
# Remove new lines from column names
data.columns = data.columns.str.replace('\n', ' ')
# Update Long Column Name
data.columns = data.columns.str.replace('DOCUMENT OTHER PROFESSIONAL CERTIFICATION CRITIERA Five Years Work Experience Teaching Excellence Professional Certifications', 'Other Experience')

# Trim all remaining columns that are strings
data['Name'] = data['Name'].str.strip()
data['Location'] = data['Location'].str.strip()
data['Title'] = data['Title'].str.strip()
data['Reports To'] = data['Reports To'].str.strip()
data['Highest Qualification'] = data['Highest Qualification'].str.strip()
data['Major'] = data['Major'].str.strip()
data['University'] = data['University'].str.strip()
data['All Qualifications from Profile'] = data['All Qualifications from Profile'].str.strip()
data['Courses Taught- Term 201510'] = data['Courses Taught- Term 201510'].str.strip()
data['MAJOR TEACHING FIELD'] = data['MAJOR TEACHING FIELD'].str.strip()
data['Other Experience'] = data['Other Experience'].str.strip()
data['Criteria'] = data['Criteria'].str.strip()

### Examining the Data

First step is to look at the head of the data.

In [3]:
data.head()

,ID,Name,Location,Grade,Title,Join Date,LWD,Type,Divison,Reports To,Highest Qualification Level,Highest Qualification,Major,University,All Qualifications from Profile,Courses Taught- Term 201510,MAJOR TEACHING FIELD,Other Experience,Criteria
0,LT9210743,Jim Hunter,London,FA,Faculty - Business,29-Aug-10,NaN,Full Time,Business,Abbas Houda,Masters,Master of Arts,International Affairs,American University of Beirut,"Bachelor of Science (Communications), Master o...",BUS2403 - Innovation & Entrepreneurship\nBUS20...,Business Administration,"10+ Years Small Business Ownership, 10+ Years ...",Industry & Teaching experience
1,LT9001784,Jose Nadal,London,FA,Faculty - Business,19-Aug-99,NaN,Full Time,Business,Abbas Houda,Masters,Master of Science,Business Info. Technology,University of Northumbria,"Bachelor of Arts (Finance), Certificate (Manag...",BUS4173 - Managerial Accounting II\nBUS4916 - ...,Business,years of teaching experience: 17 ...,Certification
2,LT9062955,Pierre Cornet,London,FA,Faculty - Business,08-Apr-07,NaN,Full Time,Business,Abbas Houda,Masters,Master of Business Administrat,Marketing,Griffith University,"Certificate (Educating/Teaching Adult), Associ...",BUS3903 - Contemporary Business\nBUS1303 - Mar...,Marketing and Strategic Management,Years of teaching experience: 15\nYears of pro...,Industry & Teaching experience
3,0,Sally Smith,London,FA,Faculty - Business,14-Aug-16,NaN,Full Time,Business,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,HRM/ Organizational Psychology,"The University of Swansea, UK","Bachelor(finance & Mgt), Masters(MBA & Masters...","HRM, Organizational behaviour, Strategic Mgt, ...",HRM,6 years professional experience + 6 years teac...,Researcher
4,0,Ned Shearer,London,Chair,Faculty - Business,14-Aug-16,NaN,Full Time,Business,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,Economics and International Business,"New York University, USA","Bachelor( Economics), Masters( Economics and I...","Micro and Macroeconomics, Statistics,Introduct...",Economics and International Business,1 year professional experience + 24 years teac...,Researcher/Curriculum Development


Initial conclusions from view the head is that there are a few columns that are quite information heavy that likely can be broken out into more meaningful columns with only one piece of information. There are also the qualifications columns that could be streamlined.

#### Unique Identifier

Firstly we should check the dataset has a column for unique identification.

In [4]:
# Check for duplicates
print(data['ID'].value_counts())

# Check for nulls
id_missing = data['ID'].isnull().sum()
print("\nNumber of missing values:", id_missing)

0            40
LT9210743     1
LT9017549     1
LT9001148     1
LT9305052     1
             ..
LT9308826     1
LT9210710     1
LT9303893     1
LT9328836     1
LT9307987     1
Name: ID, Length: 243, dtype: int64

Number of missing values: 2


Taking a first look at the columns shows the ID column has missing values. We will assign a unique id column to the dataset so we can keep track of each line.

It should be recommended that the original ids are recovered in order to link back to the original dataset, this column will be retained in order to do this in future also.

In [5]:
# Add a unique id column
u_id = data.index
data.insert(0, 'u_id', u_id)

data.head()

,u_id,ID,Name,Location,Grade,Title,Join Date,LWD,Type,Divison,Reports To,Highest Qualification Level,Highest Qualification,Major,University,All Qualifications from Profile,Courses Taught- Term 201510,MAJOR TEACHING FIELD,Other Experience,Criteria
0,0,LT9210743,Jim Hunter,London,FA,Faculty - Business,29-Aug-10,NaN,Full Time,Business,Abbas Houda,Masters,Master of Arts,International Affairs,American University of Beirut,"Bachelor of Science (Communications), Master o...",BUS2403 - Innovation & Entrepreneurship\nBUS20...,Business Administration,"10+ Years Small Business Ownership, 10+ Years ...",Industry & Teaching experience
1,1,LT9001784,Jose Nadal,London,FA,Faculty - Business,19-Aug-99,NaN,Full Time,Business,Abbas Houda,Masters,Master of Science,Business Info. Technology,University of Northumbria,"Bachelor of Arts (Finance), Certificate (Manag...",BUS4173 - Managerial Accounting II\nBUS4916 - ...,Business,years of teaching experience: 17 ...,Certification
2,2,LT9062955,Pierre Cornet,London,FA,Faculty - Business,08-Apr-07,NaN,Full Time,Business,Abbas Houda,Masters,Master of Business Administrat,Marketing,Griffith University,"Certificate (Educating/Teaching Adult), Associ...",BUS3903 - Contemporary Business\nBUS1303 - Mar...,Marketing and Strategic Management,Years of teaching experience: 15\nYears of pro...,Industry & Teaching experience
3,3,0,Sally Smith,London,FA,Faculty - Business,14-Aug-16,NaN,Full Time,Business,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,HRM/ Organizational Psychology,"The University of Swansea, UK","Bachelor(finance & Mgt), Masters(MBA & Masters...","HRM, Organizational behaviour, Strategic Mgt, ...",HRM,6 years professional experience + 6 years teac...,Researcher
4,4,0,Ned Shearer,London,Chair,Faculty - Business,14-Aug-16,NaN,Full Time,Business,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,Economics and International Business,"New York University, USA","Bachelor( Economics), Masters( Economics and I...","Micro and Macroeconomics, Statistics,Introduct...",Economics and International Business,1 year professional experience + 24 years teac...,Researcher/Curriculum Development


## Data Analysis

### Values in the Columns
We will now examine the contents of the columns in order to identify if any columns are redudant. This will be done by counting the number of unique values in each column and eliminating those where most of the column is the same information.

In [24]:
# Get total number of rows
total = len(data)
print("Total Number of Rows",total)

#Summarise the data
data.describe(include = 'O')

Total Number of Rows 284


,ID,Name,Location,Title,Join Date,LWD,Reports To,Highest Qualification Level,Highest Qualification,Major,University,All Qualifications from Profile,Courses Taught- Term 201510,MAJOR TEACHING FIELD,Other Experience,Criteria,Title_clean
count,282,284,270,284,283,11,280,278,278,262,278,274,273,225,222,284,284
unique,243,282,15,11,143,2,15,18,59,112,214,262,269,129,207,8,6
top,0,Zikida Koudou,Bristol,Faculty - Business,14-Aug-16,17-Dec-15,Ian Varley,Masters,Master of Business Administrat,Business Administration,University of Wollongong,Master of Business Administrat (Business Admin...,Faculty Trainee/MBA in Progress,General Business,More than 5 years work experience,Industry & Teaching experience,Business
freq,40,2,49,253,41,6,33,157,73,45,8,7,2,20,14,231,265


Taking a closer look at the columns with 2 or less different values.

In [7]:
# Print the counts for each of the values within the columns
print(data['Grade'].value_counts())
print(data['LWD'].value_counts())
print(data['Type'].value_counts())
print(data['Divison'].value_counts())

FA       282
Chair      2
Name: Grade, dtype: int64
17-Dec-15    6
22-Dec-15    5
Name: LWD, dtype: int64
Full Time    284
Name: Type, dtype: int64
Business    284
Name: Divison, dtype: int64


For Grade, Type and Divison the majority of the columns contain the same data, so can be removed. LWD only have two dates and is mostly null otherwise, however it may be a column useful to filtering, such as if we only want data from active staff, so this column will be kept.

In [8]:
# Drop irrelevant columns Grade, Type, Division
data = data.drop(columns=['Grade','Type','Divison'])

data.head()

,u_id,ID,Name,Location,Title,Join Date,LWD,Reports To,Highest Qualification Level,Highest Qualification,Major,University,All Qualifications from Profile,Courses Taught- Term 201510,MAJOR TEACHING FIELD,Other Experience,Criteria
0,0,LT9210743,Jim Hunter,London,Faculty - Business,29-Aug-10,NaN,Abbas Houda,Masters,Master of Arts,International Affairs,American University of Beirut,"Bachelor of Science (Communications), Master o...",BUS2403 - Innovation & Entrepreneurship\nBUS20...,Business Administration,"10+ Years Small Business Ownership, 10+ Years ...",Industry & Teaching experience
1,1,LT9001784,Jose Nadal,London,Faculty - Business,19-Aug-99,NaN,Abbas Houda,Masters,Master of Science,Business Info. Technology,University of Northumbria,"Bachelor of Arts (Finance), Certificate (Manag...",BUS4173 - Managerial Accounting II\nBUS4916 - ...,Business,years of teaching experience: 17 ...,Certification
2,2,LT9062955,Pierre Cornet,London,Faculty - Business,08-Apr-07,NaN,Abbas Houda,Masters,Master of Business Administrat,Marketing,Griffith University,"Certificate (Educating/Teaching Adult), Associ...",BUS3903 - Contemporary Business\nBUS1303 - Mar...,Marketing and Strategic Management,Years of teaching experience: 15\nYears of pro...,Industry & Teaching experience
3,3,0,Sally Smith,London,Faculty - Business,14-Aug-16,NaN,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,HRM/ Organizational Psychology,"The University of Swansea, UK","Bachelor(finance & Mgt), Masters(MBA & Masters...","HRM, Organizational behaviour, Strategic Mgt, ...",HRM,6 years professional experience + 6 years teac...,Researcher
4,4,0,Ned Shearer,London,Faculty - Business,14-Aug-16,NaN,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,Economics and International Business,"New York University, USA","Bachelor( Economics), Masters( Economics and I...","Micro and Macroeconomics, Statistics,Introduct...",Economics and International Business,1 year professional experience + 24 years teac...,Researcher/Curriculum Development


### Checking for missing values
The next step in the Data Analysis is to examine those columns that have missing values. This will be used throughout each individual examination of each column in order to address missing values.

In [9]:
# Count Missing
count_missing = data.isnull().sum()
total = len(data)

percent_missing = (count_missing/total) * 100
print(percent_missing)

u_id                                0.000000
ID                                  0.704225
Name                                0.000000
Location                            4.929577
Title                               0.000000
Join Date                           0.352113
LWD                                96.126761
Reports To                          1.408451
Highest Qualification Level         2.112676
Highest Qualification               2.112676
Major                               7.746479
University                          2.112676
All Qualifications from Profile     3.521127
Courses Taught- Term 201510         3.873239
MAJOR TEACHING FIELD               20.774648
Other Experience                   21.830986
Criteria                            0.000000
dtype: float64


### Extracting useful information from Courses Taught
The courses taught field contains multiple values of courses. Having a column with multiple pieces of information inside of it is not very useful, but previous experience of specific subjects may be a useful feature. 

Here we will transform this column into a separate table which can be used alongside the main table in the future.

In [35]:
# Split out the courses in new columns
courses_t  = data['Courses Taught- Term 201510'].str.split(',|\n', expand=True)

# Ensure the unique id is added to the new dataframe
courses_t.insert(0, 'u_id', u_id)

# Transform the new dataframe so the columns become rows
courses_t = courses_t.melt(id_vars=['u_id'], var_name='column', value_name='Course')

# Remove empty rows
courses_t = courses_t[~courses_t['Course'].isnull()]

# Trim columns and remove redudnant column
courses_t['Course'] = courses_t['Course'].str.strip()
courses_t = courses_t.drop(columns=['column'])

# Export to a new file
courses_t.to_csv('cleaned_data\courses_table.csv')
courses_t.head()

,u_id,Course
0,0,BUS2403 - Innovation & Entrepreneurship
1,1,BUS4173 - Managerial Accounting II
2,2,BUS3903 - Contemporary Business
3,3,HRM
4,4,Micro and Macroeconomics


It is possible to extract information using these columns in order to indicate past teacing experience. We will do this by finding the most common subject keywords and indicating if these keywords appear in this column for each member of staff.

In [11]:
# Convert the column to a list of courses and split by new line and space
courses_list = data['Courses Taught- Term 201510'].to_list()
courses_list_keywords = []
for x in range(0,len(courses_list)-1):
    y = str(courses_list[x])
    courses_list_keywords.append(re.split(r' |\n',y))

# Count the occurance of each of the keywords
courses_list_keywords = [x for y in courses_list_keywords for x in y]
courses_list_keywords_count = Counter(courses_list_keywords)

# Get most common keywords
common = courses_list_keywords_count.most_common(20)
items = [x[0] for x in common]

for x in items:
    if x.isalpha():
        print(x)

Business
and
Management
Accounting
for
Work
Financial
Related
Learning
Finance
Marketing
Mgt
Managers
Strategic
Busi
Project
Corporate


Using this list, we can put together some new columns that could be useful for machine learning analysis. This would be dependant on the application for this dataset, and should only be considered an example.

In [12]:
# Adding a '1' for yes where the keyword exists in the courses taught column
data['taught_business'] = np.where(data['Courses Taught- Term 201510'].str.contains('Business') |
                                   data['Courses Taught- Term 201510'].str.contains('Busi '), 1, 0)

data['taught_management'] = np.where(data['Courses Taught- Term 201510'].str.contains('Manag') |
                                     data['Courses Taught- Term 201510'].str.contains('Mgmnt') |
                                     data['Courses Taught- Term 201510'].str.contains('Mgt'), 1, 0)

data['taught_accounting'] = np.where(data['Courses Taught- Term 201510'].str.contains('Accounting'), 1, 0)

data['taught_finance'] = np.where(data['Courses Taught- Term 201510'].str.contains('Financ'), 1, 0)

data['taught_marketing'] = np.where(data['Courses Taught- Term 201510'].str.contains('Marketing'), 1, 0)

data.head()

,u_id,ID,Name,Location,Title,Join Date,LWD,Reports To,Highest Qualification Level,Highest Qualification,...,All Qualifications from Profile,Courses Taught- Term 201510,MAJOR TEACHING FIELD,Other Experience,Criteria,taught_business,taught_management,taught_accounting,taught_finance,taught_marketing
0,0,LT9210743,Jim Hunter,London,Faculty - Business,29-Aug-10,NaN,Abbas Houda,Masters,Master of Arts,...,"Bachelor of Science (Communications), Master o...",BUS2403 - Innovation & Entrepreneurship\nBUS20...,Business Administration,"10+ Years Small Business Ownership, 10+ Years ...",Industry & Teaching experience,1,1,0,0,0
1,1,LT9001784,Jose Nadal,London,Faculty - Business,19-Aug-99,NaN,Abbas Houda,Masters,Master of Science,...,"Bachelor of Arts (Finance), Certificate (Manag...",BUS4173 - Managerial Accounting II\nBUS4916 - ...,Business,years of teaching experience: 17 ...,Certification,0,1,1,0,0
2,2,LT9062955,Pierre Cornet,London,Faculty - Business,08-Apr-07,NaN,Abbas Houda,Masters,Master of Business Administrat,...,"Certificate (Educating/Teaching Adult), Associ...",BUS3903 - Contemporary Business\nBUS1303 - Mar...,Marketing and Strategic Management,Years of teaching experience: 15\nYears of pro...,Industry & Teaching experience,1,0,0,0,1
3,3,0,Sally Smith,London,Faculty - Business,14-Aug-16,NaN,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,...,"Bachelor(finance & Mgt), Masters(MBA & Masters...","HRM, Organizational behaviour, Strategic Mgt, ...",HRM,6 years professional experience + 6 years teac...,Researcher,0,1,0,0,0
4,4,0,Ned Shearer,London,Faculty - Business,14-Aug-16,NaN,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,...,"Bachelor( Economics), Masters( Economics and I...","Micro and Macroeconomics, Statistics,Introduct...",Economics and International Business,1 year professional experience + 24 years teac...,Researcher/Curriculum Development,1,0,0,0,0


### Cleaning Title
Examining the contents of the Title column gives some inconsistencies in the formatting.

In [13]:
# Print the counts for each of the values within the columns
print(data['Title'].value_counts())

Faculty - Business                       253
Faculty - Trainee                         13
Adjunct Faculty - Business                 9
Faculty- Business                          2
Faculty - Business (Admin)                 1
Faculty-Business (HR & Admin.)             1
Faculty - Business & Comp Lit Faculty      1
Faculty-Business                           1
Business, Comp & Math Faculty              1
Graduate Trainee                           1
Faculty - Business & Comp Lit.             1
Name: Title, dtype: int64


These can be corrected to give move consistent values. It is worth noting that this column is similarly not very diverse and may be considered redundant.

In [16]:
data['Title_clean'] = np.where(data['Title'].str.contains('HR') &
                               data['Title'].str.contains('Business'), 'Business (HR & Admin)',
                      np.where(data['Title'].str.contains('Admin') &
                               data['Title'].str.contains('Business'), 'Business (Admin)',
                      np.where(data['Title'].str.contains('Comp') &
                               data['Title'].str.contains('Business') &
                               data['Title'].str.contains('Math'), 'Business, Comp & Math',
                      np.where(data['Title'].str.contains('Comp Lit') &
                               data['Title'].str.contains('Business'), 'Business & Comp Lit',
                      np.where(data['Title'].str.contains('Trainee'), 'Trainee',
                      np.where(data['Title'].str.contains('Business'), 'Business', 'Other'))))))

data.head()

,u_id,ID,Name,Location,Title,Join Date,LWD,Reports To,Highest Qualification Level,Highest Qualification,...,Courses Taught- Term 201510,MAJOR TEACHING FIELD,Other Experience,Criteria,taught_business,taught_management,taught_accounting,taught_finance,taught_marketing,Title_clean
0,0,LT9210743,Jim Hunter,London,Faculty - Business,29-Aug-10,NaN,Abbas Houda,Masters,Master of Arts,...,BUS2403 - Innovation & Entrepreneurship\nBUS20...,Business Administration,"10+ Years Small Business Ownership, 10+ Years ...",Industry & Teaching experience,1,1,0,0,0,Business
1,1,LT9001784,Jose Nadal,London,Faculty - Business,19-Aug-99,NaN,Abbas Houda,Masters,Master of Science,...,BUS4173 - Managerial Accounting II\nBUS4916 - ...,Business,years of teaching experience: 17 ...,Certification,0,1,1,0,0,Business
2,2,LT9062955,Pierre Cornet,London,Faculty - Business,08-Apr-07,NaN,Abbas Houda,Masters,Master of Business Administrat,...,BUS3903 - Contemporary Business\nBUS1303 - Mar...,Marketing and Strategic Management,Years of teaching experience: 15\nYears of pro...,Industry & Teaching experience,1,0,0,0,1,Business
3,3,0,Sally Smith,London,Faculty - Business,14-Aug-16,NaN,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,...,"HRM, Organizational behaviour, Strategic Mgt, ...",HRM,6 years professional experience + 6 years teac...,Researcher,0,1,0,0,0,Business
4,4,0,Ned Shearer,London,Faculty - Business,14-Aug-16,NaN,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,...,"Micro and Macroeconomics, Statistics,Introduct...",Economics and International Business,1 year professional experience + 24 years teac...,Researcher/Curriculum Development,1,0,0,0,0,Business


### Cleaning Highest Qualification
There are several columns that refer to the qualifications on the person. These are a bit inconsistent and could do with some tidying up.

The first is the Highest Qualification Level and Highest Qualification.

In [25]:
# Print the counts for each of the values within the columns
print(data['Highest Qualification Level'].value_counts())

# Print the counts for each of the values within the columns
print(data['Highest Qualification'].value_counts())

Masters                                                                     157
Doctorate                                                                    52
Ph.D                                                                         43
Bachelor                                                                     12
Masters (Business Administration), Master of Philosophy (Business)            1
 Doctorate of Business Administration                                         1
 Ph. D. (Business Administration)                                             1
Maters of Philosophy (Economics)                                              1
Master's Degree in Public Relations and Press Office Management               1
PhD Corporate Governance and Accounting                                       1
Master of Business Administration (MBA) and MA (Economics)                    1
Bachelor (Applied Science) in Business administration( Human Resources )      1
Ph.D in Organization and Management     

Highest Qualification Level shares values with the Highest Qualification column, as well as Highest Qualification containing lots of varied values for each individual subject, despite there being the column Major to take care of this.

As shown previously, there are also some null values in the columns. We can take information from the All Qualifications from Profile column in order to attempt to backfill these null values.

Using the data in these columns we can clean them up more distinct categories.

In [29]:
# Clean up Highest Qualification Level
data['HQL_clean'] = np.where(data['Highest Qualification Level'].isna() & 
                             data['All Qualifications from Profile'].str.contains('Ph.D'),'Doctorate',
                    np.where(data['Highest Qualification Level'].isna() & 
                             data['All Qualifications from Profile'].str.contains('Master'),'Masters',
                    np.where(data['Highest Qualification Level'].isna() & 
                             data['All Qualifications from Profile'].str.contains('Bachelor'),'Bachelors', 
                    np.where(data['Highest Qualification Level'].str.contains('Master'), 'Masters',
                    np.where(data['Highest Qualification Level'].str.contains('Ph.D') |
                             data['Highest Qualification Level'].str.contains('PhD'), 'Doctorate',
                    np.where(data['Highest Qualification Level'].str.contains('MBA'), 'Masters',
                    np.where(data['Highest Qualification Level'].str.contains('Doctor') &
                            (data['Highest Qualification'].str.contains('Ph.D') |
                             data['Highest Qualification'].str.contains('philosophy')), 'Doctorate',
                    np.where(data['Highest Qualification Level'].str.contains('Doctor'), 'Doctorate',
                    np.where(data['Highest Qualification'].str.contains('Doctor'), 'Doctorate',
                    np.where(data['Highest Qualification Level'].str.contains('Bachelor'), 'Bachelors',
                             'Unknown'))))))))))

# Clean up Highest Qualification
data['HQ_clean'] = np.where(data['Highest Qualification Level'].isna() & 
                             data['All Qualifications from Profile'].str.contains('Ph.D'),'Ph.D',
                    np.where(data['Highest Qualification Level'].isna() & 
                             data['All Qualifications from Profile'].str.contains('Master'),'Masters',
                    np.where(data['Highest Qualification Level'].isna() & 
                             data['All Qualifications from Profile'].str.contains('Bachelor'),'Bachelors',
                    np.where(data['Highest Qualification'].str.contains('Master of Arts'), 'Master of Arts',
                    np.where(data['Highest Qualification'].str.contains('Master of Science'), 'Master of Science',
                    np.where(data['Highest Qualification'].str.contains('Master of Business Administrat'), 'Master of Business Administration',
                    np.where(data['Highest Qualification'].str.contains('Bachelor of Applied Science'), 'Bachelor of Applied Science',
                    np.where(data['Highest Qualification'].str.contains('Master of Commerce'), 'Master of Commerce',
                    np.where(data['Highest Qualification'].str.contains('Master of Marketing'), 'Master of Marketing',
                    np.where(data['Highest Qualification'].str.contains('Master of Business'), 'Master of Business',
                    np.where(data['Highest Qualification'].str.contains('Master of Management'), 'Master of Management',
                    np.where(data['Highest Qualification'].str.contains('Master of Law'), 'Master of Law',
                    np.where(data['Highest Qualification'].str.contains('Master of Philosophy'), 'Master of Philosophy',
                    np.where(data['Highest Qualification'].str.contains('Master of Education'), 'Master of Education',
                    np.where(data['Highest Qualification'].str.contains('Master of International Business'), 'Master of International Business',
                    np.where(data['Highest Qualification'].str.contains('Bachelor of Science'), 'Bachelor of Science',
                    np.where(data['Highest Qualification'].str.contains('Bachelor of Business Admin'), 'Bachelor of Business Administration',
                    np.where(data['Highest Qualification'].str.contains('Master of Professional Studies'), 'Master of Professional Studies',
                    np.where(data['Highest Qualification Level'].str.contains('MBA'), 'Master of Business Administration',
                    np.where(data['Highest Qualification Level'].str.contains('Master'), 'Masters',
                    np.where(data['Highest Qualification Level'].str.contains('Ph.D') |
                             data['Highest Qualification Level'].str.contains('PhD') |
                             data['Highest Qualification'].str.contains('Ph.D') |
                             data['Highest Qualification'].str.contains('PhD'), 'Ph.D',
                    np.where(data['Highest Qualification Level'].str.contains('MBA'), 'Masters',
                    np.where(data['Highest Qualification Level'].str.contains('Doctor') &
                            (data['Highest Qualification'].str.contains('Ph.D') |
                             data['Highest Qualification'].str.contains('philosophy')), 'Ph.D',
                    np.where(data['Highest Qualification Level'].str.contains('Doctor'), 'Doctorate (Other)',
                    np.where(data['Highest Qualification'].str.contains('Doctor'), 'Doctorate (Other)',
                    np.where(data['Highest Qualification Level'].str.contains('Bachelor'), 'Bachelors',   
                             'Unknown'))))))))))))))))))))))))))

print(data['HQL_clean'].value_counts())
print(data['HQ_clean'].value_counts())

Masters      168
Doctorate    102
Bachelors     13
Unknown        1
Name: HQL_clean, dtype: int64
Ph.D                                   93
Master of Business Administration      76
Masters                                40
Master of Science                      18
Doctorate (Other)                       9
Bachelor of Applied Science             9
Master of Commerce                      9
Master of Arts                          7
Master of Philosophy                    5
Master of Education                     3
Master of Management                    3
Master of Law                           2
Master of Business                      2
Bachelors                               2
Master of Marketing                     1
Master of International Business        1
Bachelor of Science                     1
Bachelor of Business Administration     1
Master of Professional Studies          1
Unknown                                 1
Name: HQ_clean, dtype: int64


Running the check again gives streamlined categories and only one remaining Unknown value. Since there is only one, we can account for the missing by filling in with the most common value (Masters).

In [32]:
data['HQL_clean'] = data['HQL_clean'].str.replace('Unknown', 'Masters', regex=False)
data['HQ_clean'] = data['HQ_clean'].str.replace('Unknown', 'Masters', regex=False)

data.head()

,u_id,ID,Name,Location,Title,Join Date,LWD,Reports To,Highest Qualification Level,Highest Qualification,...,Other Experience,Criteria,taught_business,taught_management,taught_accounting,taught_finance,taught_marketing,Title_clean,HQL_clean,HQ_clean
0,0,LT9210743,Jim Hunter,London,Faculty - Business,29-Aug-10,NaN,Abbas Houda,Masters,Master of Arts,...,"10+ Years Small Business Ownership, 10+ Years ...",Industry & Teaching experience,1,1,0,0,0,Business,Masters,Master of Arts
1,1,LT9001784,Jose Nadal,London,Faculty - Business,19-Aug-99,NaN,Abbas Houda,Masters,Master of Science,...,years of teaching experience: 17 ...,Certification,0,1,1,0,0,Business,Masters,Master of Science
2,2,LT9062955,Pierre Cornet,London,Faculty - Business,08-Apr-07,NaN,Abbas Houda,Masters,Master of Business Administrat,...,Years of teaching experience: 15\nYears of pro...,Industry & Teaching experience,1,0,0,0,1,Business,Masters,Master of Business Administration
3,3,0,Sally Smith,London,Faculty - Business,14-Aug-16,NaN,Abbas Houda,Ph.D,Ph.D in HRM/ Organizational pyschology,...,6 years professional experience + 6 years teac...,Researcher,0,1,0,0,0,Business,Doctorate,Ph.D
4,4,0,Ned Shearer,London,Faculty - Business,14-Aug-16,NaN,Tony Emmanuel,Ph.D,Ph.D in Economics and International Business,...,1 year professional experience + 24 years teac...,Researcher/Curriculum Development,1,0,0,0,0,Business,Doctorate,Ph.D


The major column can then be used to store the subject. This could be cleaned and extracted from the various available columns and streamlined in a similar way to above to indicate the staffs main subject expertise.

### All Qualifications from Profile

This column shows the qualification and the subject in the best format but contains multiple pieces of information. It could be used to verify or populate the highest qualification available, although there are some discrepancies between the columns.

It can also be transformed similarly to the courses taught column in order to extract information that could be helpful, demonstrated below.

In [34]:
# Split out the courses in new columns
qualifs = data['All Qualifications from Profile'].str.split(',', expand=True)

# Ensure the unique id is added to the new dataframe
qualifs.insert(0, 'u_id', u_id)

# Transform the new dataframe so the columns become rows
qualifs_t = qualifs.melt(id_vars=['u_id'], var_name='column', value_name='qualif')

# Remove empty rows
qualifs_t = qualifs_t[~qualifs_t['qualif'].isnull()]

# Split out qualification from subject
qualifs_t[['Qualification','Major']] = qualifs_t['qualif'].str.split('(', 1, expand=True)

# Clean up
qualifs_t['Major'] = qualifs_t['Major'].str.replace('))', 'nnn', regex=False)
qualifs_t['Major'] = qualifs_t['Major'].str.replace(')', '', regex=False)
qualifs_t['Major'] = qualifs_t['Major'].str.replace('nnn', ')', regex=False)
qualifs_t['Major'] = qualifs_t['Major'].str.replace('(', '', regex=False)
qualifs_t['Qualification'] = qualifs_t['Qualification'].str.strip()
qualifs_t['Major'] = qualifs_t['Major'].str.strip()
qualifs_t = qualifs_t.drop(columns=['column', 'qualif'])

# Export to a new file
qualifs_t.to_csv('cleaned_data\qualifications_table.csv')
qualifs_t.head()

,u_id,Qualification,Major
0,0,Bachelor of Science,Communications
1,1,Bachelor of Arts,Finance
2,2,Certificate,Educating/Teaching Adult
3,3,Bachelor,finance & Mgt
4,4,Bachelor,Economics
